## Hypothesis and Inference

In [41]:
import math
import random

In [42]:
def normal_approximation_to_binormal(n, p):
    '''finds mu and sigmal corresponding to a Binomial(n, p)'''
    mu = p * n
    sigma = math.sqrt(p * (1-p) * n)
    return mu, sigma

def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    '''find approximate inverse using binary search'''
    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)    
    low_z = -10.0
    hi_z  = 10.0
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2
        mid_p = normal_cdf(mid_z)        
        if mid_p < p:
            # midpoint is still too low, search above it
            low_z = mid_z
        elif mid_p > p:
            # midpoint is still too high, search below it
            hi_z = mid_z
        else:
            break
            
    return mid_z       

def normal_cdf(x,mu=0, sigma=1):
    return (1 + math.erf((x-mu)/math.sqrt(2)/sigma))/2

# the normal cdf is the probability the variable is below threshold
normal_probability_below = normal_cdf

def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

def normal_upper_bound(probability, mu=0, sigma=1):
    '''returns the z for which P(Z >= z) = probability'''
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    '''returns the z for which P(Z >= z) = probability'''
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    '''returns the symmetric (about the mean) bounds that
    contain the specified probability'''
    tail_probability = (1 - probability) / 2
    
    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound
    

    

if __name__ == "__main__":
    
    # normal approximation to binomial test
    mean, std_dev = normal_approximation_to_binormal(150, 0.9)
    print('B({}, {}) = N({}, {})'.format(150, 0.9, mean, std_dev))
    
    #
    mu_0, sigma_0 = normal_approximation_to_binormal(1000, 0.5)
    print('B({}, {}) = N({}, {})'.format(1000, 0.5, mu_0, sigma_0))
    
    # 95% bounds based on assumption p is 0.5
    lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)
    print('p={}, low={}, high={}'.format(0.95, lo, hi))
    
    # actual mu and sigma based on p = 0.55
    mu_1, sigma_1 = normal_approximation_to_binormal(1000, 0.55)
    print('p={}, mu={}, sigma={}'.format(0.55, mu_1, sigma_1))
    
    # type 2 error means that we fail to rejec the null hypothesis
    # which will happen when X is still in our original interval
    type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
    power = 1 - type_2_probability
    print('p = ', power)
    
    hi = normal_upper_bound(0.95, mu_0, sigma_0)
    type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
    power_01 = 1 - type_2_probability
    print('p = ', power_01)
    
    

B(150, 0.9) = N(135.0, 3.674234614174767)
B(1000, 0.5) = N(500.0, 15.811388300841896)
p=0.95, low=469.01026640487555, high=530.9897335951244
p=0.55, mu=550.0, sigma=15.732132722552274
p =  0.8865480012953671
p =  0.9363794803307173


### P-values

In [43]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is what's greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is what's less than x
        return 2 * normal_probability_below(x, mu, sigma)

def run_experiment():
    '''flip a fair coin 1000 times, True = heads, False = tails'''
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment):
    '''using the 5% significance levels'''
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531
#--------------------------------------------------------------------
def estimated_parameters(N, n):
    p = n/N
    sigma = math.sqrt(p * (1-p)/N)
    return p, sigma

def a_b_test_statistics(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A)/ math.sqrt(sigma_A **2 + sigma_B **2)

def B(alpha, beta):
    '''a normalizing constant so that the total probability is 1'''
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    if x <= 0 or x >= 1:              # no weight outside of [0, 1]
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)
#---------------------------------------------------------------------------        
    
if __name__ == "__main__":
    
    p = two_sided_p_value(529.5, mu_0, sigma_0)
    print('p-value = ', p)
    
    # alternatively
    extreme_value_count = 0
    for _ in range(100000):
        num_heads = sum(1 if random.random() < 0.5 else 0
                       for _ in range(1000))
        if num_heads >= 530 or num_heads <= 470:
            extreme_value_count += 1
    
    print('p-value (alt)', extreme_value_count/100000)  
    
    p1 = two_sided_p_value(531.5, mu_0, sigma_0)
    print('p-value (p1) = ', p1)
    
    upper_p_value = normal_probability_above
    lower_p_value = normal_probability_below
    
    p2 = upper_p_value(524.5, mu_0, sigma_0)
    print('p-value (p2) = ', p2)
    p3 = upper_p_value(526.5, mu_0, sigma_0)
    print('p-value (p3) = ', p3)

p-value =  0.06207721579598857
p-value (alt) 0.06155
p-value (p1) =  0.046345287837786575
p-value (p2) =  0.06062885772582083
p-value (p3) =  0.04686839508859242


In [44]:
    random.seed(0)
    experiments = [run_experiment() for _ in range(1000)]
    num_rejection = len([experiment 
                         for experiment in experiments 
                         if reject_fairness(experiment)])
    
    print('number of rejections = ', num_rejection)

number of rejections =  46
